# Data

### MOT15

In [ ]:
from pathlib import Path

data_dir = Path("/home/username/Downloads/MOT15")
!ls $data_dir

In [ ]:
import pandas as pd

train_mot15_df = pd.read_csv(Path(data_dir, "annotation_train.csv"))
train_mot15_df.head()

In [ ]:
import albumentations as A

# IMAGE_SIZE = (416, 416)
IMAGE_SIZE = (1088, 608)
# IMAGE_SIZE = (608, 608)
train_transform = A.Compose(
    [
        A.Resize(height=IMAGE_SIZE[1], width=IMAGE_SIZE[0], p=1.0),
    ], 
    p=1.0, 
    bbox_params=A.BboxParams(
        format="pascal_voc",
        min_area=0,
        min_visibility=0,
        label_fields=["person_ids"]
    )
)

val_transform = A.Compose(
    [
        A.Resize(height=IMAGE_SIZE[1], width=IMAGE_SIZE[0], p=1.0),
    ],
    p=1.0,
    bbox_params=A.BboxParams(
        format="pascal_voc",
        min_area=0,
        min_visibility=0,
        label_fields=["person_ids"]
    )
)

In [ ]:
from src.train_dataset import TrainDataset
%load_ext autoreload
%autoreload 2

train_mot15_dir = Path(data_dir, "train")
train_dataset = TrainDataset(train_mot15_dir, train_mot15_df, train_transform)

In [ ]:
def get_color(idx):
    idx = idx * 3
    color = ((37 * idx) % 255 / 255, (17 * idx) % 255 / 255, (29 * idx) % 255 / 255)
    return color

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
import cv2

image, centers, person_ids = train_dataset[25].values()
for center, person_id in zip(centers.tolist(), person_ids.tolist()):
    x, y = map(int, center)
    color = get_color(person_id)
    cv2.circle(
        image, (x, y), 20, color, 2, lineType=cv2.LINE_AA
    )

plt.imshow(image)
plt.show()

# Modeling

In [ ]:
# import gdown

# gdown.download(
#     "https://drive.google.com/uc?id=1SFOhg_vos_xSYHLMTDGFVZBYjo8cr2fG", str(Path(data_dir, "crowdhuman_dla34.pth")), quiet=False
# )
# gdown.download(
#     "https://drive.google.com/uc?id=1iqRQjsG9BawIl8SlFomMg5iwkb6nqSpi", str(Path(data_dir, "fairmot_dla34.pth")), quiet=False
# )

In [ ]:
import sys
sys.path.append("../..")
from person_detection.centernet.src.pose_dla_dcn import get_pose_net as get_dla_dcn
from src.trainer import Trainer
%load_ext autoreload
%autoreload 2

num_layers = 34
heads = {"hm": 1, "wh": 4, "reg": 2, "id": 128}
head_conv = 256
net = get_dla_dcn(num_layers, heads, head_conv)
# model_path = Path("/home/jasseur/Downloads/crowdhuman_dla34.pth)"
model_path = Path("/home/jasseur/Downloads/fairmot_dla34.pth")
trainer = Trainer(
    net, image_size=IMAGE_SIZE, device="cuda", checkpoint_dir="checkpoint",
    nID=train_dataset.nID, model_path=model_path
)

In [ ]:
trainer.fit(train_dataset, train_dataset, batch_size=4, epochs=2)

In [ ]:
trainer.eval(train_dataset, batch_size=4)